In [6]:
import nltk                    
from operator import eq 
import math

def OpinosisGraph(Z, G, PRI):  
    for i in range(len(Z)):  
        for j in range(len(Z[i])):  
            LABEL = Z[i][j]
            PID = j
            SID = i
            if LABEL in G:
                PRI[LABEL].append((SID, PID))
            else:
                G[LABEL] = list()
                PRI[LABEL] = [(SID, PID)]
            if j > 0 and LABEL not in G[Z[i][j-1]]:
                G[Z[i][j-1]].append(Z[i][j])
                
def VSN(node):   
    global PRI1
    sigma_vsn = 15
    ele = PRI1[node]
    sum = 0
    for i in range(len(ele)):
        sum = sum + ele[i][1]
    return (sum/len(ele)) <= sigma_vsn

def VEN(node):  
    prohibit = ['and', 'but', 'for', 'nor', 'or', 'so', 'yet', ',', '.', '!', '?']
    if(node[0] in prohibit):
        return True
    return False

def collapsible(node): 
    if(node[1] == 'VERB'):
        return True
    else: 
        return False

def valid_sent(sent):                        
    global no_winner
    
    all = ['VERB','.','ADJ','PRON','NOUN','ADV','PRON','CONJ','PRT','ADP','NUM','DET','X']
    count = dict()
    for i in sent:
        if i not in count:
            count[i] = 1
        else:
            count[i] += 1
    for i in count:
        if i[1] in ['ADJ','ADV','NOUN', 'PRON'] and count[i] > 1:
            return False
    if no_winner == 0:
        return True
    #the first kind of valid sentence
    state = 0
    for i in sent:
        if state == 0 and i[1] == 'NOUN':
            state = 1
        if state == 1 and i[1] == 'VERB':
            state = 2
        if state == 2 and i[1] == 'ADJ':
            state = 3
    if state == 3:
        return True
    #the second kind of valid sentence
    state = 0
    for i in sent:
        if state == 0 and i[1] == 'ADJ':
            state = 2
        if state == 1 and i[1] == 'DET':  
            state = 2
        if state == 2 and i[1] == 'VERB':
            state = 3
    if state == 3:
        return True
    #the third kind of valid sentence
    state = 0
    for i in sent:
        if state == 0 and i[1] == 'ADV':
            state = 1
        if state == 1 and i[1] == 'ADJ':  
            state = 2
        if state == 2 and i[1] == 'NOUN':
            state = 3
    if state == 3:
        return True
    #the fourth kind of valid sentence
    state = 0
    for i in sent:
        if state == 0 and i[1] == 'ADV':
            state = 2
        #if state == 1 and i[1] == 'VERB':  #?
            #state = 2
        if state == 2 and i[1] == 'NOUN':
            state = 3
    if state == 3:
        return True
    return False

def valid_sent2(sent):    
    prohibit = {'VERB':['VERB'],      
        'NOUN':['ADJ','PRON','NOUN'],
        'ADJ':['VERB','PRON','ADV'],
        'ADV':['ADJ','NOUN','PRON','ADV'],
        'PRON':['ADJ','PRON','NOUN'], #I,she
        'CONJ':['CONJ','PRT','ADP'], #and
        'PRT':['CONJ','PRT','ADP'], # 's
        '.':['VERB','.','ADJ','PRON','NOUN','ADV','PRON','CONJ','PRT','ADP','NUM','DET','X'], #punctuation
        'ADP':['CONJ','PRT','ADP'], #prep+介詞
        'NUM':[],
        'DET':['CONJ','PRT','ADP'], #to
        'X':[]
      }
    for i in range(0,len(sent)-1):
        if(sent[i+1][1] in prohibit[sent[i][1]]):
              return False
    buf = []
    for i in sent:
        if(i not in buf):
            buf.append(i)
        elif(i[1] == 'ADJ' or i[1] == 'ADV' or i[1] == 'NOUN'):
            return False
    return True

def check_inter(PRIa,PRIb):  
    PRIc = []
    sigma_gap = 4
    for first in PRIa:
        for second in PRIb:
            if(first[0] == second[0] and abs(first[1]-second[1]) <= sigma_gap):
                PRIc.append(second)
    return PRIc

def common_elements(list1, list2): 
    count = 0
    for element in list1:
        if element in list2:
            count= count+1
    return count
                                            
def jaccard_fail(a,b):
    sigma_j = 0.2
    inter = common_elements(a,b)
    if ((inter/(len(a)+len(b)-inter)) >= sigma_j):
        return True
    if abs(inter-len(a)) < 3 or abs(inter-len(b)) < 3:
        return True
    return False

def eliminate_dup(clist): 
    buf = []
    first = True
    for ele in clist:
        temp = []
        similar = False
        if(first):
            buf.append(ele)
            first = False
        else:
            for seen in buf:
                if jaccard_fail(seen[0],ele[0]):
                    similar = True
                    if ele[1]>seen[1]:                           
                        temp.append(seen)
            if len(temp) != 0:  
                for seen in temp:
                    buf.remove(seen)
                buf.append(ele)
            elif similar == False:
                buf.append(ele)  
    return buf

def find_avg_score(clist):    
    if len(clist)==0:
        return 0
    sum = 0
    for ele in clist:
        sum = sum + ele[1]
    return sum/len(clist)

def stitch(Canchor,CC):    
    if len(CC) == 0:
        return Canchor
    winner = CC[0]
    for ele in CC:
        if ele[1] > winner[1]:
            winner = ele
    ans = []
    return (Canchor+winner[0])

def Sort_Tuple(tup):  
    lst = len(tup)  
    for i in range(0, lst):  
          
        for j in range(0, lst-i-1):  
            if (tup[j][1] < tup[j + 1][1]):  
                temp = tup[j]  
                tup[j]= tup[j + 1]  
                tup[j + 1]= temp  
    return tup  
    


def check_verb(sent):
    for word,pos in sent:
        if pos == 'VERB':
            return True
    return False

def find_sum(C):             
    sortC = Sort_Tuple(C)   
    if(len(C)==0):
        return "FAIL"
    winner = C[0]
    ans1 = ""
    for words in winner[0]:
        ans1 = ans1 + words[0] + " "
    ans1 = ans1[0].upper() + ans1[1:(len(ans1)-1)]
    return ans1
    
def Traverse(clist,vk,score,PRI_overlap,sent,length): 
    global ending
    #print(score)
    global G1
    global PRI1
    global sigma_r          
    redundancy = len(PRI_overlap)
    if score > 40:
        ending = 1
    if ending == 1:
        return
    if length>15:        
        return
    if (redundancy  >= sigma_r):  
        if (VEN(vk)):
            if (valid_sent(sent)):
                finalscore = score/length
                clist.append((sent,finalscore))
                return 
        for vn in G1[vk]:
            PRI_new = check_inter(PRI_overlap,PRI1[vn])  
            redundancy = len(PRI_new)
            new_sent = sent+[vn]
            L = length + 1
            new_score = score + redundancy #+ math.log2(L) 
            if (collapsible(vn)):
                Canchor = new_sent
                tmp = []
                for vx in G1[vn]:
                    Traverse(tmp,vx,0,PRI_new,[vx],L)
                    CC = eliminate_dup(tmp) 
                    avg_CC_score = find_avg_score(CC)
                    finalscore = new_score + avg_CC_score
                    stitch_sent = stitch(Canchor,CC)  
                    if(valid_sent(stitch_sent)):
                        clist.append((stitch_sent,finalscore))
            else:
                Traverse(clist,vn,new_score,PRI_new,new_sent,L)

def first_lower(s):    
    if s[0] == "I":
        return s
    else:
        return s[0].lower() + s[1:]
        
def read_text(file_name):   
    redundant = '<br />'
    review = []
    with open(file_name,'r') as file:
        for article in file:
            article = article.replace(redundant," ")
            sentences = nltk.sent_tokenize(article)    
            for sentence in sentences:
                sentence = first_lower(sentence)
                review.append(nltk.pos_tag(nltk.word_tokenize(sentence),tagset='universal'))           
    file.close()
    return review



In [7]:
negative_word = open('negative-words.txt','r')
negative = set()
for i in range(4784):
    a = negative_word.readline()
    negative.add(a[:-2])
positive_word = open('positive-words.txt','r')
positive = set()
for i in range(2006):
    a = positive_word.readline()
    positive.add(a[:-2])

In [8]:
def read_text2(file_name):   
    redundant = '<br />'
    review = []
    with open(file_name,'r') as file:
        for article in file:
            article = article.replace(redundant," ")
            sentences = nltk.sent_tokenize(article)    
            for sentence in sentences:
                sentence = first_lower(sentence)
                sentence = sentence.split(' ')
                review.append(sentence)     
    file.close()
    return review

In [11]:
import nltk                                                     #predict based on the text and the summary
import os
filenames = [f for f in os.listdir('.') if os.path.isfile(f)]
filenames.remove('negative-words.txt')
filenames.remove('positive-words.txt')
new_filenames = []
for i in filenames:
    if i[-3:] == 'txt':
        new_filenames.append(i)
result = []
for name in new_filenames:   
    ending = 0
    
    a,b = name.split('.')
    if b == 'txt':
        neutral = 0
        sigma_r = 1
        no_winner = 0
        Z1 = read_text(name)
        G1 = dict()
        PRI1 = dict()
        OpinosisGraph(Z1, G1, PRI1)  
        candidates = []
        for keys in G1:
            if(VSN(keys)):
                clist = []
                sum_sent = [keys]
                score = 0
                Traverse(clist,keys,score,PRI1[keys],sum_sent,1)
                candidates = candidates + clist
        C = eliminate_dup(candidates)
        if len(C) == 0:
            sigma_r = 1
            Z1 = read_text(name)
            G1 = dict()
            PRI1 = dict()
            OpinosisGraph(Z1, G1, PRI1)  
            candidates = []
            for keys in G1:
                if(VSN(keys)):
                    clist = []
                    sum_sent = [keys]
                    score = 0
                    Traverse(clist,keys,score,PRI1[keys],sum_sent,1)
                    candidates = candidates + clist
            C = eliminate_dup(candidates)
            break
        summary = find_sum(C)
        
        #interpolation with original data
        for word in summary[0]:
            if word in negative:
                neutral -= 1
            elif word in positive:
                neutral += 2
        a,b = name.split('.')
        if b == 'txt':
            Z1 = read_text(name)
        for i in Z1:
            for word in i:
                if word[0] in negative and word[1] in ['ADJ','ADV']:
                    neutral -= 1/len(Z1)
                elif word[0] in positive and word[1] in ['ADJ','ADV']:
                    neutral += 2/len(Z1)
        result.append([name, neutral])
        
correct = 0
for i in range(len(result)):
    a = result[i][0].split('_')
    a = a[1].split('.')
    if a[0] in ['0','1','2','3','4']:
        result[i].append("corrct prediction" if (result[i][1] <= -3) else "wrong prediction")
    else: 
        result[i].append("corrct prediction" if (result[i][1] > -3) else "wrong prediction")
    if result[i][2] == "corrct prediction":
        correct += 1
    print(result[i])  
print("Prediction Accuracy(based on the text and the summary): ",correct / len(result))

['00_3.txt', 0, 'wrong prediction']
['00_9.txt', -0.1111111111111111, 'corrct prediction']
['01_1.txt', -0.18181818181818182, 'wrong prediction']
['01_7.txt', 0, 'corrct prediction']
['02_1.txt', -0.14285714285714285, 'wrong prediction']
['02_9.txt', 0, 'corrct prediction']
['03_10.txt', 0, 'corrct prediction']
['03_4.txt', 0, 'wrong prediction']
['04_10.txt', 0.14285714285714285, 'corrct prediction']
['04_4.txt', 0, 'wrong prediction']
['05_10.txt', 0.1111111111111111, 'corrct prediction']
['05_3.txt', 0.06666666666666667, 'wrong prediction']
['06_1.txt', 0, 'wrong prediction']
['06_7.txt', 0, 'corrct prediction']
['07_3.txt', -0.26666666666666666, 'wrong prediction']
['07_7.txt', 0, 'corrct prediction']
['08_4.txt', 0.0, 'wrong prediction']
['08_7.txt', 0, 'corrct prediction']
['09_1.txt', -0.3333333333333333, 'wrong prediction']
['09_9.txt', -0.125, 'corrct prediction']
['10_2.txt', 0, 'wrong prediction']
['10_9.txt', 0.25, 'corrct prediction']
['11_3.txt', -0.16666666666666666, 'wr

In [12]:
filenames = [f for f in os.listdir('.') if os.path.isfile(f)]   #predict based on only the text
filenames.remove('negative-words.txt')
filenames.remove('positive-words.txt')
new_filenames = []
for i in filenames:
    if i[-3:] == 'txt':
        new_filenames.append(i)
result = []
#new_filenames = ['30_10.txt']
for name in new_filenames:         
    a,b = name.split('.')
    if b == 'txt':
        sigma_r = 2
        Z1 = read_text2(name)
    neutral = 0
    for i in Z1:
        for word in i:
            if word in negative:
                neutral -= 1
            elif word in positive:
                neutral += 2
    result.append([name, neutral])

correct = 0
for i in range(len(result)):
    if result[i][0][3: 5] == '10':
        result[i].append("correct prediction" if (result[i][1] > -10) else "wrong prediction")
    elif result[i][0][3] in ['0','1','2','3','4']:
        result[i].append("correct prediction" if (result[i][1] < -10) else "wrong prediction")
    else: 
        result[i].append("correct prediction" if (result[i][1] > -10) else "wrong prediction")
    if result[i][2] == "correct prediction" :
        correct += 1
    print(result[i])
print("Prediction Accuracy(based on only the text): ",correct / len(result))

['00_3.txt', -1, 'wrong prediction']
['00_9.txt', 11, 'correct prediction']
['01_1.txt', -8, 'wrong prediction']
['01_7.txt', 4, 'correct prediction']
['02_1.txt', -3, 'wrong prediction']
['02_9.txt', 0, 'correct prediction']
['03_10.txt', 9, 'correct prediction']
['03_4.txt', 9, 'wrong prediction']
['04_10.txt', 4, 'correct prediction']
['04_4.txt', 4, 'wrong prediction']
['05_10.txt', 7, 'correct prediction']
['05_3.txt', 20, 'wrong prediction']
['06_1.txt', 9, 'wrong prediction']
['06_7.txt', -4, 'correct prediction']
['07_3.txt', -1, 'wrong prediction']
['07_7.txt', 8, 'correct prediction']
['08_4.txt', 0, 'wrong prediction']
['08_7.txt', 12, 'correct prediction']
['09_1.txt', -2, 'wrong prediction']
['09_9.txt', 9, 'correct prediction']
['10_2.txt', 12, 'wrong prediction']
['10_9.txt', 2, 'correct prediction']
['11_3.txt', -7, 'wrong prediction']
['11_9.txt', 3, 'correct prediction']
['12_1.txt', 2, 'wrong prediction']
['12_9 .txt', 10, 'correct prediction']
['13_2.txt', 3, 'wrong